<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterer-preproc

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Saving kaggle.json to kaggle (1).json
 90% 185M/206M [00:01<00:00, 150MB/s]
100% 206M/206M [00:01<00:00, 158MB/s]


In [ ]:
!unzip /content/betterer-preproc.zip

In [6]:
!cp -r /content/archive ~/.kaggle/input/

In [10]:
!ls ~/.kaggle/input

test  train


In [24]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

 26% 9.00M/34.1M [00:00<00:00, 50.3MB/s]
100% 34.1M/34.1M [00:00<00:00, 135MB/s] 


In [ ]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

# PRE PROCESS
.JPG TO .NPY

In [52]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/content/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [97]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, f"{input_folder}{i}.npy")
            np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [78]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


6161

In [79]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 1092 2560 1792


In [80]:
import imgaug.augmenters as iaa

# Define the augmenter
augmenter = iaa.Sequential([
    iaa.Multiply((0.8, 1.2)),
    iaa.LinearContrast((0.8, 1.2)),
])

augmented_images = []

for path in moderateDem:
    path = os.path.join("/kaggle/working/train", path)
    image = np.load(path)

    for _ in range(20):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [57]:
len(augmented_images)

1040

In [60]:
import cv2
import os


output_directory = "/kaggle/working/train"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
    np.save(output_path, image)


print("Augmented images saved successfully.")


Augmented images saved successfully.


In [81]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

6161

In [ ]:
!ls -l /kaggle/working/train

In [83]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 1092 2560 1792


In [84]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    2560
3    1792
1    1092
0     717
Name: count, dtype: int64

In [85]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [86]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [87]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[1 3 0 ... 3 3 1]
[1 0 2 ... 3 3 2]


In [88]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(4928, 208, 176, 3) (1233, 208, 176, 3) (4928,) (1233,)


In [89]:
print(test_labels)
test_labels_encoded

[1 0 2 ... 3 3 2]


array([1, 0, 2, ..., 3, 3, 2])

In [90]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define CNN model
model = models.Sequential([
    layers.Input(shape=train_images[0].shape),  # Input layer with shape of input data
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4)  # Number of classes
])

# Define precision and recall metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# Define F1 score function
def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

# Compile the model
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', tf.keras.metrics.SparseCategoricalAccuracy()])

# Train the model
history = model.fit(train_images, train_labels_encoded, epochs=10, validation_data=(test_images, test_labels_encoded))

Epoch 1/10
154/154 [==============================] - 7s 36ms/step - loss: 9.2265 - accuracy: 0.6297 - sparse_categorical_accuracy: 0.6297 - val_loss: 0.5957 - val_accuracy: 0.7397 - val_sparse_categorical_accuracy: 0.7397
Epoch 2/10
154/154 [==============================] - 5s 31ms/step - loss: 0.4731 - accuracy: 0.7971 - sparse_categorical_accuracy: 0.7971 - val_loss: 0.4711 - val_accuracy: 0.8110 - val_sparse_categorical_accuracy: 0.8110
Epoch 3/10
154/154 [==============================] - 5s 34ms/step - loss: 0.3166 - accuracy: 0.8787 - sparse_categorical_accuracy: 0.8787 - val_loss: 0.4207 - val_accuracy: 0.8337 - val_sparse_categorical_accuracy: 0.8337
Epoch 4/10
154/154 [==============================] - 5s 34ms/step - loss: 0.1650 - accuracy: 0.9407 - sparse_categorical_accuracy: 0.9407 - val_loss: 0.2627 - val_accuracy: 0.8978 - val_sparse_categorical_accuracy: 0.8978
Epoch 5/10
154/154 [==============================] - 5s 33ms/step - loss: 0.0834 - accuracy: 0.9728 - spars

In [91]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)

y_prob = tf.nn.softmax(y_prob).numpy()

y_true = np.eye(4)[test_labels_encoded]

# Compute ROC AUC for each class
roc_auc = roc_auc_score(y_true, y_prob, average='macro')

print("ROC AUC:", roc_auc)


39/39 [==============================] - 0s 9ms/step
ROC AUC: 0.9944331537311125


In [92]:
# Save the model
model.save("my_cnn_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [98]:
import os
import numpy as np
directory = '/content/Alzheimer_s Dataset/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1279

In [100]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [101]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/Alzheimer_s Dataset/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [102]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [103]:
labels = df["Class"]
labels = label_encoder.transform(labels)
# Verify the encoded labels
print(labels)

[0 0 0 ... 3 3 3]


In [104]:
model.evaluate(images, labels)

40/40 [==============================] - 0s 11ms/step - loss: 2.5724 - accuracy: 0.5778 - sparse_categorical_accuracy: 0.5778


[2.572382926940918, 0.577795147895813, 0.577795147895813]